In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
import numpy as np 
import pandas as pd 
import seaborn as sns
!pip install gensim spacy nltk
import re 
import gensim
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
data= pd.read_csv('/content/drive/My Drive/Cleaned Data/Sarcasm.csv') 

In [4]:
import numpy as np
max_sequence_length = np.max(data['Number of Words in tweet'])
print("max length is:", max_sequence_length)

max length is: 56


In [5]:
text = data['cleaned_tweet']
label = data['sarcasm']

In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(text.astype(str))
sequences = tokenizer.texts_to_sequences(text.astype(str))

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))


from keras_preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Found 55395 unique tokens.
Shape of data tensor: (26095, 56)


In [7]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(label))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (26095, 2)


In [8]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

# Arabert

In [9]:
!pip install bert-for-tf2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=4899dab07aff01d38813a7252b0c45ac23b210afdaebcf9060422cb8355d3f45
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=ea01aafe5fd768d452f927503244a8bc23b45b7fb2042ad3bdc49469fea462e5
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=680c9f26cd743643113a550166ae07cf22fbc376eb83d1b5400d73a0e3904bf1
  Sto

In [10]:
import numpy as np
import math
import re
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [11]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
from transformers import AutoTokenizer, AutoModel

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [12]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv01",do_lower_case=False)
def encode_sentence(sent):
    tokenized = arabert_tokenizer.tokenize(sent)
    return arabert_tokenizer.convert_tokens_to_ids(tokenized)

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

# BiLSTM-BiLSTM Models

In [ ]:
# BiLSTM Layers with Adam(1e-4), (Relu + Sigmoid)

from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='sigmoid')
])


model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10)

predictions= model.predict(x_test)
predictions=np.argmax(predictions, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions))
print('Recall: %.3f' % recall_score(y_test, predictions))
print('Precision: %.3f' % precision_score(y_test, predictions))
print('F1 Score: %.3f' % f1_score(y_test, predictions))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 22s 25ms/step - loss: 0.4485 - accuracy: 0.8245
Epoch 2/10
612/612 [==============================] - 15s 25ms/step - loss: 0.2558 - accuracy: 0.8998
Epoch 3/10
612/612 [==============================] - 16s 27ms/step - loss: 0.1346 - accuracy: 0.9540
Epoch 4/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0774 - accuracy: 0.9755
Epoch 5/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0519 - accuracy: 0.9854
Epoch 6/10
612/612 [==============================] - 18s 30ms/step - loss: 0.0381 - accuracy: 0.9897
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0314 - accuracy: 0.9904
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0260 - accuracy: 0.9915
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0230 - accuracy: 0.9922
Epoch 10/10
204/204 [==============================] - 3s 8ms/step
The Confusion Matrix is [[

In [ ]:
# BiLSTM Layers, Adam(1e-4), (Relu+Softmax)
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model1.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy1= model1.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy1)

predictions1= model1.predict(x_test)
predictions1=np.argmax(predictions1, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions1)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions1))
print('Recall: %.3f' % recall_score(y_test, predictions1))
print('Precision: %.3f' % precision_score(y_test, predictions1))
print('F1 Score: %.3f' % f1_score(y_test, predictions1))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 24ms/step - loss: 0.4580 - accuracy: 0.8220
Epoch 2/10
612/612 [==============================] - 15s 24ms/step - loss: 0.2867 - accuracy: 0.8840
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.1648 - accuracy: 0.9414
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0957 - accuracy: 0.9689
Epoch 5/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0612 - accuracy: 0.9822
Epoch 6/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0431 - accuracy: 0.9881
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0354 - accuracy: 0.9900
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0280 - accuracy: 0.9917
Epoch 9/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0255 - accuracy: 0.9919
Epoch 10/10
204/204 [==============================] - 4s 13ms/step - loss: 0.4377 - accuracy

# Activation Functions Tuning

In [ ]:
#BiLSTM Layers, Adam(1e-3), (softplus + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='softplus'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy2= model2.evaluate(x_test, y_test)
print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)
predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('Recall: %.3f' % recall_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 32s 37ms/step - loss: 0.4376 - accuracy: 0.8250
Epoch 2/10
612/612 [==============================] - 22s 35ms/step - loss: 0.2283 - accuracy: 0.9154
Epoch 3/10
612/612 [==============================] - 24s 39ms/step - loss: 0.1258 - accuracy: 0.9596
Epoch 4/10
612/612 [==============================] - 24s 40ms/step - loss: 0.0899 - accuracy: 0.9727
Epoch 5/10
612/612 [==============================] - 23s 38ms/step - loss: 0.0535 - accuracy: 0.9854
Epoch 6/10
612/612 [==============================] - 21s 35ms/step - loss: 0.0401 - accuracy: 0.9885
Epoch 7/10
612/612 [==============================] - 23s 38ms/step - loss: 0.0273 - accuracy: 0.9911
Epoch 8/10
612/612 [==============================] - 24s 39ms/step - loss: 0.0196 - accuracy: 0.9932
Epoch 9/10
612/612 [==============================] - 22s 37ms/step - loss: 0.0159 - accuracy: 0.9926
Epoch 10/10
204/204 [==============================] - 4s 12ms/step - loss: 0.4706 - accuracy

In [ ]:
#2 BiLSTM Layers  1e-3, Adam (softsign + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model4 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='softsign'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model4.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model4.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy4= model4.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy4)

predictions4= model4.predict(x_test)
predictions4=np.argmax(predictions4, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions4)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions4))
print('Recall: %.3f' % recall_score(y_test, predictions4))
print('Precision: %.3f' % precision_score(y_test, predictions4))
print('F1 Score: %.3f' % f1_score(y_test, predictions3))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 23s 25ms/step - loss: 0.3716 - accuracy: 0.8475
Epoch 2/10
612/612 [==============================] - 15s 24ms/step - loss: 0.1443 - accuracy: 0.9497
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0529 - accuracy: 0.9842
Epoch 4/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0266 - accuracy: 0.9908
Epoch 5/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0175 - accuracy: 0.9917
Epoch 6/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0144 - accuracy: 0.9927
Epoch 7/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0126 - accuracy: 0.9932
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0119 - accuracy: 0.9935
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0165 - accuracy: 0.9924
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4827 - accuracy:

In [ ]:
#2 BiLSTM Layers  1e-3, Adam (tanh + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model5 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model5.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model5.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy5= model5.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy5)

predictions5= model5.predict(x_test)
predictions5=np.argmax(predictions5, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions5)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions5))
print('Recall: %.3f' % recall_score(y_test, predictions5))
print('Precision: %.3f' % precision_score(y_test, predictions5))
print('F1 Score: %.3f' % f1_score(y_test, predictions5))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 21s 25ms/step - loss: 0.3632 - accuracy: 0.8489
Epoch 2/10
612/612 [==============================] - 15s 25ms/step - loss: 0.1305 - accuracy: 0.9543
Epoch 3/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0422 - accuracy: 0.9857
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0210 - accuracy: 0.9909
Epoch 5/10
612/612 [==============================] - 16s 25ms/step - loss: 0.0183 - accuracy: 0.9910
Epoch 6/10
612/612 [==============================] - 17s 28ms/step - loss: 0.0176 - accuracy: 0.9921
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0133 - accuracy: 0.9926
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0147 - accuracy: 0.9926
Epoch 9/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0127 - accuracy: 0.9935
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.6222 - accuracy:

In [ ]:
#2 BiLSTM Layers, Adam(1e-3), (selu + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model6 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='selu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model6.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model6.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy5= model6.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy5)
predictions6= model6.predict(x_test)
predictions6=np.argmax(predictions6, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions6)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions6))
print('Recall: %.3f' % recall_score(y_test, predictions6))
print('Precision: %.3f' % precision_score(y_test, predictions6))
print('F1 Score: %.3f' % f1_score(y_test, predictions6))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 24ms/step - loss: 0.3674 - accuracy: 0.8512
Epoch 2/10
612/612 [==============================] - 15s 24ms/step - loss: 0.1346 - accuracy: 0.9504
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0457 - accuracy: 0.9857
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0211 - accuracy: 0.9913
Epoch 5/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0150 - accuracy: 0.9925
Epoch 6/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0138 - accuracy: 0.9923
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0135 - accuracy: 0.9924
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0134 - accuracy: 0.9931
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0146 - accuracy: 0.9924
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.6118 - accuracy:

In [ ]:
#2 BiLSTM Layers  1e-3, Adam (elu + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model7 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model7.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model7.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy7= model7.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy7)

predictions7= model7.predict(x_test)
predictions7=np.argmax(predictions7, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions7)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions7))
print('Recall: %.3f' % recall_score(y_test, predictions7))
print('Precision: %.3f' % precision_score(y_test, predictions7))
print('F1 Score: %.3f' % f1_score(y_test, predictions7))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 22s 27ms/step - loss: 0.3662 - accuracy: 0.8503
Epoch 2/10
612/612 [==============================] - 16s 26ms/step - loss: 0.1353 - accuracy: 0.9508
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0451 - accuracy: 0.9852
Epoch 4/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0224 - accuracy: 0.9909
Epoch 5/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0154 - accuracy: 0.9924
Epoch 6/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0140 - accuracy: 0.9929
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0132 - accuracy: 0.9931
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0120 - accuracy: 0.9937
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0120 - accuracy: 0.9930
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.6903 - accuracy:

In [ ]:
#2 BiLSTM Layers  1e-3, Adam (exponential + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model8 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='exponential'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model8.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model8.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy8= model8.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy8)


predictions8= model8.predict(x_test)
predictions8=np.argmax(predictions8, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions8)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions8))
print('Recall: %.3f' % recall_score(y_test, predictions8))
print('Precision: %.3f' % precision_score(y_test, predictions8))
print('F1 Score: %.3f' % f1_score(y_test, predictions8))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 24ms/step - loss: 0.4508 - accuracy: 0.8250
Epoch 2/10
612/612 [==============================] - 15s 24ms/step - loss: 0.2354 - accuracy: 0.9140
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.1213 - accuracy: 0.9606
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0634 - accuracy: 0.9804
Epoch 5/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0345 - accuracy: 0.9879
Epoch 6/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0223 - accuracy: 0.9907
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0199 - accuracy: 0.9921
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0166 - accuracy: 0.9927
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0133 - accuracy: 0.9937
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.8908 - accuracy:

# Learning Rate & Optimizers Hyperparameter Tuning on (Relu + Softmax)

In [ ]:
#BiLSTM Layers, Adam(1e-3), (Relu + Softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model3.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model3.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy3= model3.evaluate(x_test, y_test)
print('test accuracy is: ', accuracy3)

predictions3= model3.predict(x_test)
predictions3=np.argmax(predictions3, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions3)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions3))
print('Recall: %.3f' % recall_score(y_test, predictions3))
print('Precision: %.3f' % precision_score(y_test, predictions3))
print('F1 Score: %.3f' % f1_score(y_test, predictions3))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 25ms/step - loss: 0.4091 - accuracy: 0.8234
Epoch 2/10
612/612 [==============================] - 15s 24ms/step - loss: 0.2369 - accuracy: 0.9011
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.1741 - accuracy: 0.9298
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0919 - accuracy: 0.9647
Epoch 5/10
612/612 [==============================] - 17s 28ms/step - loss: 0.0440 - accuracy: 0.9848
Epoch 6/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0229 - accuracy: 0.9915
Epoch 7/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0180 - accuracy: 0.9924
Epoch 8/10
612/612 [==============================] - 20s 33ms/step - loss: 0.0136 - accuracy: 0.9936
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0129 - accuracy: 0.9942
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.7381 - accuracy:

In [ ]:
#BiLSTM Layers, Adam(1e-2), (Relu+Softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model9 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model9.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-2),
              metrics=['accuracy'])

model9.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy9= model9.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy9)


predictions9= model9.predict(x_test)
predictions9=np.argmax(predictions9, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions9)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions9))
print('Recall: %.3f' % recall_score(y_test, predictions9))
print('Precision: %.3f' % precision_score(y_test, predictions9))
print('F1 Score: %.3f' % f1_score(y_test, predictions9))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 24ms/step - loss: 0.4496 - accuracy: 0.8215
Epoch 2/10
612/612 [==============================] - 15s 24ms/step - loss: 0.3407 - accuracy: 0.8758
Epoch 3/10
612/612 [==============================] - 17s 28ms/step - loss: 0.3255 - accuracy: 0.8815
Epoch 4/10
612/612 [==============================] - 17s 28ms/step - loss: 0.3236 - accuracy: 0.8912
Epoch 5/10
612/612 [==============================] - 17s 27ms/step - loss: 0.3540 - accuracy: 0.8688
Epoch 6/10
612/612 [==============================] - 18s 29ms/step - loss: 0.3322 - accuracy: 0.8713
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.3159 - accuracy: 0.8733
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.2909 - accuracy: 0.8813
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.2757 - accuracy: 0.8831
Epoch 10/10
204/204 [==============================] - 6s 15ms/step - loss: 0.4225 - accuracy

In [ ]:
#BiLSTM Layers, Adam(1e-1), (Relu+Softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model10 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model10.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-1),
              metrics=['accuracy'])

model10.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy10= model10.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy10)

predictions10= model10.predict(x_test)
predictions10=np.argmax(predictions10, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions10))
cm = confusion_matrix(y_test, predictions10)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions10))
print('F1 Score: %.3f' % f1_score(y_test, predictions10))
print('Precision: %.3f' % precision_score(y_test, predictions10))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 21s 24ms/step - loss: 0.4927 - accuracy: 0.8158
Epoch 2/10
612/612 [==============================] - 18s 29ms/step - loss: 0.4748 - accuracy: 0.8188
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.4747 - accuracy: 0.8188
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.4752 - accuracy: 0.8188
Epoch 5/10
612/612 [==============================] - 15s 25ms/step - loss: 0.4747 - accuracy: 0.8188
Epoch 6/10
612/612 [==============================] - 15s 24ms/step - loss: 0.4747 - accuracy: 0.8188
Epoch 7/10
612/612 [==============================] - 15s 24ms/step - loss: 0.4744 - accuracy: 0.8188
Epoch 8/10
612/612 [==============================] - 15s 25ms/step - loss: 0.4750 - accuracy: 0.8188
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.4747 - accuracy: 0.8188
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4636 - accuracy:

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Model 2: 2 BiLSTM Layers with Softmax loss function at the dense layer, 1e-4, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model11 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model11.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy'])

model11.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy11= model11.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy11)

predictions11= model11.predict(x_test)
predictions11=np.argmax(predictions11, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions11))
cm = confusion_matrix(y_test, predictions11)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions11))
print('F1 Score: %.3f' % f1_score(y_test, predictions11))
print('Precision: %.3f' % precision_score(y_test, predictions11))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 25s 30ms/step - loss: 0.4633 - accuracy: 0.8209
Epoch 2/10
612/612 [==============================] - 19s 31ms/step - loss: 0.3737 - accuracy: 0.8436
Epoch 3/10
612/612 [==============================] - 19s 31ms/step - loss: 0.3278 - accuracy: 0.8653
Epoch 4/10
612/612 [==============================] - 17s 28ms/step - loss: 0.2983 - accuracy: 0.8802
Epoch 5/10
612/612 [==============================] - 19s 31ms/step - loss: 0.2748 - accuracy: 0.8911
Epoch 6/10
612/612 [==============================] - 17s 28ms/step - loss: 0.2536 - accuracy: 0.9016
Epoch 7/10
612/612 [==============================] - 16s 26ms/step - loss: 0.2327 - accuracy: 0.9098
Epoch 8/10
612/612 [==============================] - 13s 20ms/step - loss: 0.2138 - accuracy: 0.9193
Epoch 9/10
612/612 [==============================] - 17s 29ms/step - loss: 0.1944 - accuracy: 0.9275
Epoch 10/10
204/204 [==============================] - 7s 14ms/step - loss: 0.3580 - accuracy

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers with Softmax loss function at the dense layer, 1e-3, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model12 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model12.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-3),
              metrics=['accuracy'])

model12.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy12= model12.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy12)

predictions12= model12.predict(x_test)
predictions12=np.argmax(predictions12, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions12))
cm = confusion_matrix(y_test, predictions12)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions12))
print('F1 Score: %.3f' % f1_score(y_test, predictions12))
print('Precision: %.3f' % precision_score(y_test, predictions12))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 30s 31ms/step - loss: 0.4128 - accuracy: 0.8318
Epoch 2/10
612/612 [==============================] - 19s 30ms/step - loss: 0.2490 - accuracy: 0.9062
Epoch 3/10
612/612 [==============================] - 17s 28ms/step - loss: 0.1541 - accuracy: 0.9447
Epoch 4/10
612/612 [==============================] - 19s 31ms/step - loss: 0.0998 - accuracy: 0.9648
Epoch 5/10
612/612 [==============================] - 20s 33ms/step - loss: 0.0705 - accuracy: 0.9769
Epoch 6/10
612/612 [==============================] - 18s 30ms/step - loss: 0.0537 - accuracy: 0.9836
Epoch 7/10
612/612 [==============================] - 17s 29ms/step - loss: 0.0409 - accuracy: 0.9869
Epoch 8/10
612/612 [==============================] - 19s 31ms/step - loss: 0.0323 - accuracy: 0.9900
Epoch 9/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0271 - accuracy: 0.9911
Epoch 10/10
204/204 [==============================] - 4s 12ms/step - loss: 0.7195 - accuracy

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers with Softmax loss function at the dense layer, 1e-2, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model13 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model13.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-2),
              metrics=['accuracy'])

model13.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy13= model13.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy13)

predictions13= model13.predict(x_test)
predictions13=np.argmax(predictions13, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions13))
cm = confusion_matrix(y_test, predictions13)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions13))
print('F1 Score: %.3f' % f1_score(y_test, predictions13))
print('Precision: %.3f' % precision_score(y_test, predictions13))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 21s 20ms/step - loss: 0.4564 - accuracy: 0.8200
Epoch 2/10
612/612 [==============================] - 12s 20ms/step - loss: 0.3393 - accuracy: 0.8788
Epoch 3/10
612/612 [==============================] - 14s 22ms/step - loss: 0.2360 - accuracy: 0.9220
Epoch 4/10
612/612 [==============================] - 14s 22ms/step - loss: 0.1788 - accuracy: 0.9438
Epoch 5/10
612/612 [==============================] - 12s 20ms/step - loss: 0.1465 - accuracy: 0.9579
Epoch 6/10
612/612 [==============================] - 14s 23ms/step - loss: 0.1224 - accuracy: 0.9632
Epoch 7/10
612/612 [==============================] - 14s 24ms/step - loss: 0.1126 - accuracy: 0.9694
Epoch 8/10
612/612 [==============================] - 17s 28ms/step - loss: 0.0997 - accuracy: 0.9733
Epoch 9/10
612/612 [==============================] - 18s 29ms/step - loss: 0.0926 - accuracy: 0.9751
Epoch 10/10
204/204 [==============================] - 5s 11ms/step - loss: 0.6114 - accuracy

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers with Softmax loss function at the dense layer, 1e-1, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model14 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model14.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-1),
              metrics=['accuracy'])

model14.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy14= model14.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy14)

predictions14= model14.predict(x_test)
predictions14=np.argmax(predictions14, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions14))
cm = confusion_matrix(y_test, predictions14)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions14))
print('F1 Score: %.3f' % f1_score(y_test, predictions14))
print('Precision: %.3f' % precision_score(y_test, predictions14))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 25s 30ms/step - loss: 0.7577 - accuracy: 0.8094
Epoch 2/10
612/612 [==============================] - 16s 27ms/step - loss: 0.5525 - accuracy: 0.8186
Epoch 3/10
612/612 [==============================] - 17s 28ms/step - loss: 0.6187 - accuracy: 0.8106
Epoch 4/10
612/612 [==============================] - 13s 21ms/step - loss: 0.5502 - accuracy: 0.8211
Epoch 5/10
612/612 [==============================] - 12s 19ms/step - loss: 0.5761 - accuracy: 0.8179
Epoch 6/10
612/612 [==============================] - 12s 19ms/step - loss: 0.6016 - accuracy: 0.8165
Epoch 7/10
612/612 [==============================] - 12s 19ms/step - loss: 0.5756 - accuracy: 0.8212
Epoch 8/10
612/612 [==============================] - 12s 19ms/step - loss: 0.5903 - accuracy: 0.8203
Epoch 9/10
612/612 [==============================] - 11s 19ms/step - loss: 0.5814 - accuracy: 0.8192
Epoch 10/10
204/204 [==============================] - 5s 12ms/step - loss: 0.4968 - accuracy

In [14]:
#BiLSTM Layers, Nadam(1e-3), (Relu + Softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model22 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model22.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Nadam(1e-3),
              metrics=['accuracy'])

model22.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy22= model22.evaluate(x_test, y_test)
print('test accuracy is: ', accuracy22)

predictions22= model22.predict(x_test)
predictions22=np.argmax(predictions22, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions22)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions22))
print('Recall: %.3f' % recall_score(y_test, predictions22))
print('Precision: %.3f' % precision_score(y_test, predictions22))
print('F1 Score: %.3f' % f1_score(y_test, predictions22))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 26s 25ms/step - loss: 0.3877 - accuracy: 0.8409
Epoch 2/10
612/612 [==============================] - 15s 25ms/step - loss: 0.1644 - accuracy: 0.9431
Epoch 3/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0605 - accuracy: 0.9815
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0269 - accuracy: 0.9898
Epoch 5/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0177 - accuracy: 0.9909
Epoch 6/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0144 - accuracy: 0.9933
Epoch 7/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0211 - accuracy: 0.9901
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0122 - accuracy: 0.9928
Epoch 9/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0110 - accuracy: 0.9930
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.7893 - accuracy:

In [15]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [16]:
#BiLSTM Layers, Adagrad(1e-3), (Relu + Softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model23 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model23.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adagrad(1e-3),
              metrics=['accuracy'])

model23.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy23= model23.evaluate(x_test, y_test)
print('test accuracy is: ', accuracy23)

predictions23= model23.predict(x_test)
predictions23=np.argmax(predictions23, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions23)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions23))
print('Recall: %.3f' % recall_score(y_test, predictions23))
print('Precision: %.3f' % precision_score(y_test, predictions23))
print('F1 Score: %.3f' % f1_score(y_test, predictions23))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 19ms/step - loss: 0.6226 - accuracy: 0.8200
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5061 - accuracy: 0.8236
Epoch 3/10
612/612 [==============================] - 13s 21ms/step - loss: 0.4712 - accuracy: 0.8236
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4707 - accuracy: 0.8236
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4695 - accuracy: 0.8236
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4693 - accuracy: 0.8236
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4683 - accuracy: 0.8236
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4686 - accuracy: 0.8236
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4688 - accuracy: 0.8236
Epoch 10/10
204/204 [==============================] - 3s 8ms/step - loss: 0.4782 - accuracy:

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [18]:
#BiLSTM Layers, Adadelta(1e-3), (Relu + Softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model24 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model24.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adadelta(1e-3),
              metrics=['accuracy'])

model24.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy24= model24.evaluate(x_test, y_test)
print('test accuracy is: ', accuracy24)

predictions24= model24.predict(x_test)
predictions24=np.argmax(predictions24, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions24)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions24))
print('Recall: %.3f' % recall_score(y_test, predictions24))
print('Precision: %.3f' % precision_score(y_test, predictions24))
print('F1 Score: %.3f' % f1_score(y_test, predictions24))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 19ms/step - loss: 0.6918 - accuracy: 0.5858
Epoch 2/10
612/612 [==============================] - 11s 19ms/step - loss: 0.6830 - accuracy: 0.8177
Epoch 3/10
612/612 [==============================] - 12s 19ms/step - loss: 0.6731 - accuracy: 0.8218
Epoch 4/10
612/612 [==============================] - 12s 19ms/step - loss: 0.6617 - accuracy: 0.8218
Epoch 5/10
612/612 [==============================] - 13s 21ms/step - loss: 0.6476 - accuracy: 0.8218
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.6308 - accuracy: 0.8218
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.6112 - accuracy: 0.8218
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5904 - accuracy: 0.8218
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5682 - accuracy: 0.8218
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.5374 - accuracy:

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [20]:
#BiLSTM Layers, Adamax(1e-3), (Relu + Softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model25 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model25.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adamax(1e-3),
              metrics=['accuracy'])

model25.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy25= model25.evaluate(x_test, y_test)
print('test accuracy is: ', accuracy25)

predictions25= model25.predict(x_test)
predictions25=np.argmax(predictions25, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions25)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions25))
print('Recall: %.3f' % recall_score(y_test, predictions25))
print('Precision: %.3f' % precision_score(y_test, predictions25))
print('F1 Score: %.3f' % f1_score(y_test, predictions25))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 19ms/step - loss: 0.4607 - accuracy: 0.8163
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.3763 - accuracy: 0.8315
Epoch 3/10
612/612 [==============================] - 11s 19ms/step - loss: 0.2952 - accuracy: 0.8780
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2282 - accuracy: 0.9138
Epoch 5/10
612/612 [==============================] - 11s 19ms/step - loss: 0.1689 - accuracy: 0.9402
Epoch 6/10
612/612 [==============================] - 13s 21ms/step - loss: 0.1243 - accuracy: 0.9579
Epoch 7/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0927 - accuracy: 0.9707
Epoch 8/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0656 - accuracy: 0.9795
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0524 - accuracy: 0.9842
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.3694 - accuracy:

# Batch Size Tuning

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers , Batch Size(10)
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model15 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model15.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model15.fit(x_train, y_train, batch_size=10, epochs=10)

accuracy15= model15.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy15)

predictions15= model15.predict(x_test)
predictions15=np.argmax(predictions15, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions15)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions15))
print('Recall: %.3f' % recall_score(y_test, predictions15))
print('Precision: %.3f' % precision_score(y_test, predictions15))
print('F1 Score: %.3f' % f1_score(y_test, predictions15))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1958/1958 [==============================] - 49s 21ms/step - loss: 0.3927 - accuracy: 0.8418
Epoch 2/10
1958/1958 [==============================] - 40s 21ms/step - loss: 0.1779 - accuracy: 0.9339
Epoch 3/10
1958/1958 [==============================] - 41s 21ms/step - loss: 0.0695 - accuracy: 0.9768
Epoch 4/10
1958/1958 [==============================] - 55s 28ms/step - loss: 0.0301 - accuracy: 0.9892
Epoch 5/10
1958/1958 [==============================] - 56s 28ms/step - loss: 0.0195 - accuracy: 0.9909
Epoch 6/10
1958/1958 [==============================] - 40s 21ms/step - loss: 0.0146 - accuracy: 0.9926
Epoch 7/10
1958/1958 [==============================] - 41s 21ms/step - loss: 0.0133 - accuracy: 0.9928
Epoch 8/10
1958/1958 [==============================] - 41s 21ms/step - loss: 0.0116 - accuracy: 0.9937
Epoch 9/10
1958/1958 [==============================] - 41s 21ms/step - loss: 0.0109 - accuracy: 0.9940
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers , Batch Size(40)
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model16 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model16.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model16.fit(x_train, y_train, batch_size=40, epochs=10)

accuracy16= model16.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy16)

predictions16= model16.predict(x_test)
predictions16=np.argmax(predictions16, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions16)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions16))
print('Recall: %.3f' % recall_score(y_test, predictions16))
print('Precision: %.3f' % precision_score(y_test, predictions16))
print('F1 Score: %.3f' % f1_score(y_test, predictions16))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


490/490 [==============================] - 16s 22ms/step - loss: 0.3972 - accuracy: 0.8366
Epoch 2/10
490/490 [==============================] - 11s 23ms/step - loss: 0.1848 - accuracy: 0.9316
Epoch 3/10
490/490 [==============================] - 11s 22ms/step - loss: 0.0727 - accuracy: 0.9776
Epoch 4/10
490/490 [==============================] - 11s 23ms/step - loss: 0.0376 - accuracy: 0.9887
Epoch 5/10
490/490 [==============================] - 11s 23ms/step - loss: 0.0206 - accuracy: 0.9909
Epoch 6/10
490/490 [==============================] - 12s 24ms/step - loss: 0.0150 - accuracy: 0.9937
Epoch 7/10
490/490 [==============================] - 11s 22ms/step - loss: 0.0143 - accuracy: 0.9932
Epoch 8/10
490/490 [==============================] - 11s 22ms/step - loss: 0.0134 - accuracy: 0.9926
Epoch 9/10
490/490 [==============================] - 11s 22ms/step - loss: 0.0131 - accuracy: 0.9934
Epoch 10/10
204/204 [==============================] - 4s 8ms/step - loss: 0.6663 - accuracy:

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers , Batch Size(60)
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model17 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model17.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model17.fit(x_train, y_train, batch_size=60, epochs=10)

accuracy17= model17.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy17)

predictions17= model17.predict(x_test)
predictions17=np.argmax(predictions17, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions17)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions17))
print('Recall: %.3f' % recall_score(y_test, predictions17))
print('Precision: %.3f' % precision_score(y_test, predictions17))
print('F1 Score: %.3f' % f1_score(y_test, predictions17))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


327/327 [==============================] - 13s 24ms/step - loss: 0.3976 - accuracy: 0.8362
Epoch 2/10
327/327 [==============================] - 8s 23ms/step - loss: 0.1609 - accuracy: 0.9417
Epoch 3/10
327/327 [==============================] - 8s 23ms/step - loss: 0.0611 - accuracy: 0.9821
Epoch 4/10
327/327 [==============================] - 8s 24ms/step - loss: 0.0332 - accuracy: 0.9890
Epoch 5/10
327/327 [==============================] - 8s 24ms/step - loss: 0.0167 - accuracy: 0.9919
Epoch 6/10
327/327 [==============================] - 8s 24ms/step - loss: 0.0140 - accuracy: 0.9925
Epoch 7/10
327/327 [==============================] - 8s 24ms/step - loss: 0.0126 - accuracy: 0.9929
Epoch 8/10
327/327 [==============================] - 8s 24ms/step - loss: 0.0150 - accuracy: 0.9925
Epoch 9/10
327/327 [==============================] - 8s 24ms/step - loss: 0.0146 - accuracy: 0.9923
Epoch 10/10
204/204 [==============================] - 3s 8ms/step - loss: 0.6326 - accuracy: 0.8990


In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers , Batch Size(80)
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model18 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model18.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model18.fit(x_train, y_train, batch_size=80, epochs=10)

accuracy18= model18.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy18)

predictions18= model18.predict(x_test)
predictions18=np.argmax(predictions18, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions18)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions18))
print('Recall: %.3f' % recall_score(y_test, predictions18))
print('Precision: %.3f' % precision_score(y_test, predictions18))
print('F1 Score: %.3f' % f1_score(y_test, predictions18))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


245/245 [==============================] - 11s 26ms/step - loss: 0.4364 - accuracy: 0.8254
Epoch 2/10
245/245 [==============================] - 6s 26ms/step - loss: 0.2250 - accuracy: 0.9167
Epoch 3/10
245/245 [==============================] - 6s 26ms/step - loss: 0.0966 - accuracy: 0.9682
Epoch 4/10
245/245 [==============================] - 6s 26ms/step - loss: 0.0483 - accuracy: 0.9861
Epoch 5/10
245/245 [==============================] - 6s 26ms/step - loss: 0.0290 - accuracy: 0.9908
Epoch 6/10
245/245 [==============================] - 6s 26ms/step - loss: 0.0189 - accuracy: 0.9931
Epoch 7/10
245/245 [==============================] - 6s 26ms/step - loss: 0.0152 - accuracy: 0.9938
Epoch 8/10
245/245 [==============================] - 6s 26ms/step - loss: 0.0184 - accuracy: 0.9916
Epoch 9/10
245/245 [==============================] - 6s 26ms/step - loss: 0.0133 - accuracy: 0.9932
Epoch 10/10
204/204 [==============================] - 4s 9ms/step - loss: 0.5782 - accuracy: 0.9076


# Epochs Tuning 

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers, Epoch(25)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model19 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model19.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model19.fit(x_train, y_train, batch_size=32, epochs=25)

accuracy19= model19.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy19)

predictions19= model19.predict(x_test)
predictions19=np.argmax(predictions19, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions19)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions19))
print('Recall: %.3f' % recall_score(y_test, predictions19))
print('Precision: %.3f' % precision_score(y_test, predictions19))
print('F1 Score: %.3f' % f1_score(y_test, predictions19))

Epoch 1/25


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 23ms/step - loss: 0.4141 - accuracy: 0.8335
Epoch 2/25
612/612 [==============================] - 14s 22ms/step - loss: 0.2150 - accuracy: 0.9227
Epoch 3/25
612/612 [==============================] - 14s 23ms/step - loss: 0.1146 - accuracy: 0.9630
Epoch 4/25
612/612 [==============================] - 14s 23ms/step - loss: 0.0613 - accuracy: 0.9814
Epoch 5/25
612/612 [==============================] - 14s 23ms/step - loss: 0.0332 - accuracy: 0.9878
Epoch 6/25
612/612 [==============================] - 14s 22ms/step - loss: 0.0224 - accuracy: 0.9912
Epoch 7/25
612/612 [==============================] - 14s 22ms/step - loss: 0.0185 - accuracy: 0.9917
Epoch 8/25
612/612 [==============================] - 14s 22ms/step - loss: 0.0152 - accuracy: 0.9929
Epoch 9/25
612/612 [==============================] - 14s 22ms/step - loss: 0.0114 - accuracy: 0.9939
Epoch 10/25
612/612 [==============================] - 14s 22ms/step - loss: 0.0109 - accurac

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers, Epoch(50)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model20 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model20.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model20.fit(x_train, y_train, batch_size=32, epochs=50)

accuracy20= model20.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy20)

predictions20= model20.predict(x_test)
predictions20=np.argmax(predictions20, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions20)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions20))
print('Recall: %.3f' % recall_score(y_test, predictions20))
print('Precision: %.3f' % precision_score(y_test, predictions20))
print('F1 Score: %.3f' % f1_score(y_test, predictions20))

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 22ms/step - loss: 0.4322 - accuracy: 0.8297
Epoch 2/50
612/612 [==============================] - 14s 24ms/step - loss: 0.2285 - accuracy: 0.9137
Epoch 3/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0980 - accuracy: 0.9686
Epoch 4/50
612/612 [==============================] - 14s 22ms/step - loss: 0.0461 - accuracy: 0.9855
Epoch 5/50
612/612 [==============================] - 14s 22ms/step - loss: 0.0237 - accuracy: 0.9908
Epoch 6/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0164 - accuracy: 0.9926
Epoch 7/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0123 - accuracy: 0.9925
Epoch 8/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0107 - accuracy: 0.9935
Epoch 9/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0137 - accuracy: 0.9923
Epoch 10/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0158 - accurac

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Model 2: 2 BiLSTM Layers, Epoch(75)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model21 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(EMB_DIM,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model21.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model21.fit(x_train, y_train, batch_size=32, epochs=75)

accuracy21= model21.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy21)

predictions21= model21.predict(x_test)
predictions21=np.argmax(predictions21, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions21)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions21))
print('Recall: %.3f' % recall_score(y_test, predictions21))
print('Precision: %.3f' % precision_score(y_test, predictions21))
print('F1 Score: %.3f' % f1_score(y_test, predictions21))

Epoch 1/75
612/612 [==============================] - 31s 33ms/step - loss: 0.4056 - accuracy: 0.8354
Epoch 2/75
612/612 [==============================] - 14s 23ms/step - loss: 0.1841 - accuracy: 0.9349
Epoch 3/75
612/612 [==============================] - 14s 23ms/step - loss: 0.0724 - accuracy: 0.9786
Epoch 4/75
612/612 [==============================] - 14s 23ms/step - loss: 0.0319 - accuracy: 0.9887
Epoch 5/75
612/612 [==============================] - 14s 23ms/step - loss: 0.0182 - accuracy: 0.9918
Epoch 6/75
612/612 [==============================] - 14s 22ms/step - loss: 0.0137 - accuracy: 0.9930
Epoch 7/75
612/612 [==============================] - 14s 22ms/step - loss: 0.0112 - accuracy: 0.9939
Epoch 8/75
612/612 [==============================] - 14s 23ms/step - loss: 0.0092 - accuracy: 0.9941
Epoch 9/75
612/612 [==============================] - 14s 22ms/step - loss: 0.0088 - accuracy: 0.9944
Epoch 10/75
612/612 [==============================] - 14s 22ms/step - loss: 0.008